## 볼린저 밴드 투자 전략
1. 이동 평균선 생성 : 데이터의 20개의 평균 값
2. 상단 밴드 생성 : 이동 평균선 + (2 * 20개의 데이터의 표준편차)
3. 하단 밴드 생성 : 이동 평균선 - (2 * 20개의 데이터의 표준편차)
4. 가격이 하단 밴드보다 낮은 경우 매수
5. 가격이 상단 밴드보다 높은 경우 매도

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
df = pd.read_csv('../../csv/AAPL.csv', index_col='Date')
df.head(1)

In [ ]:
# 결측치, 무한대 데이터를 제거 
flag = df.isin([np.nan, np.inf, -np.inf]).any(axis=1)

In [ ]:
df = df.loc[~flag, ['Adj Close']]

In [ ]:
df.iloc[0:20, 0].mean()

In [ ]:
## 이동평균선 컬럼을 생성 값들은 결측치로 대입
df['center'] = np.nan

In [ ]:
df.head(3)

In [ ]:
for i in range(20,len(df)+1):
    mean_data = df.iloc[i-20:i, 0].mean()
    df.iloc[i-1, 1] = mean_data

In [ ]:
df.iloc[18:23,]

In [ ]:
## rolling(n) : n만큼 데이터의 개수를 그룹
df['center2'] = df['Adj Close'].rolling(20).mean()

In [ ]:
df.iloc[18:24]

In [ ]:
# 상단 밴드, 하단 밴드 생성 
# 상단 밴드 = 이동 평균선  + (2 * 20개 데이터의 표준편차)
df['ub'] = \
    df['center'] + (2 * df['Adj Close'].rolling(20).std())
# 하단 밴드 = 이동 평균선 - (2 * 20개의 데이터의 표준편차)
df['lb'] = \
    df['center'] - (2 * df['Adj Close'].rolling(20).std())

In [ ]:
df.iloc[18:23, ]

In [ ]:
# index를 시계열로 변경
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

In [ ]:
x = df.tail(100).index
price_y = df.tail(100)['Adj Close']
ub_y = df.tail(100)['ub']
lb_y = df.tail(100)['lb']

plt.figure(figsize=(14, 8))
plt.plot(x, price_y)
plt.plot(x, ub_y)
plt.plot(x, lb_y)
plt.show()

In [ ]:
df[['Adj Close', 'ub', 'lb']].plot()

In [ ]:
# 투자 기간 선택 
start = '2010-01-01'

In [ ]:
# start는 시계열로 변경 
start = datetime.strptime(start, '%Y-%m-%d')

In [ ]:
test_df = df.loc[start:,]

In [ ]:
test_df.tail()

In [ ]:
# 구매 상태를 확인하는 컬럼을 생성 
test_df['trade'] = ""

### 보유 내역 추가 
- 조건식 
    - 상단밴드보다 수정 주가가 높거나 같은 경우 
        - 현재 보유상태라면 
            - 매도 (trade = "")
        - 보유 상태가 아니라면
            - 유지 (trade = "")
    - 상단밴드보다 낮고 하단밴드보다 높은 경우 
        - 현재 보유 상태라면 
            - 유지 (trade = "buy")
        - 보유 상태가 아니라면
            - 유지 (trade = "")
    - 하단 밴드보다 낮거나 같은 경우
        - 현재 보유 상태라면
            - 유지 (trade = "buy")
        - 보유 상태가 아니라면
            - 매수 (trade = "buy")

In [48]:
for i in test_df.index:
    # i가 의미하는것은? test_df의 index 값
    # print(i)
    # 수정 주가가 상단밴드보다 높거나 같은 경우
    if test_df.loc[i, 'Adj Close'] >=  test_df.loc[i, 'ub']:
        # 현재 보유중이라면 -> trade를 ""로 변경
        # 전날의 trade가 buy라면 -> 매도
        if test_df.shift().loc[i, 'trade'] == 'buy':
            test_df.loc[i, 'trade'] = ""
        # 보유중이 아니라면
        else:
            test_df.loc[i, 'trade'] = ""
    # 하단밴드보다 수정 주가가 낮거나 같은 경우 
    elif test_df.loc[i, 'Adj Close'] <= test_df.loc[i, 'lb']:
        # 현재 보유중이라면 -> trade를 "buy"로 변경
        if test_df.shift().loc[i, 'trade'] == "buy":
            test_df.loc[i, 'trade'] = "buy"
        # 현재 보유중이 아니라면 -> 매수
        else:
            test_df.loc[i, 'trade'] = "buy"
    # 수정 주가가 밴드 사이에 있을때 
    else:
        # 현재 보유 상태라면 -> 유지 trade를 "buy"
        if test_df.shift().loc[i, 'trade'] == 'buy':
            test_df.loc[i, 'trade'] = "buy"
        # 보유 상태가 아니라면 -> 유지 trade를 ""
        else:
            test_df.loc[i, 'trade'] = ""

In [49]:
test_df['trade'].value_counts()

trade
       1439
buy     945
Name: count, dtype: int64